In [ ]:
install.packages(c("tidyverse", "afex", "emmeans"))

In [ ]:
library('afex')
library("tidyverse")
library('emmeans')
set_sum_contrasts()

require(parallel)
(nc <- detectCores()) # number of cores
cl <- makeCluster(rep("localhost", nc))


# figure 4

In [4]:
df_pr_recall <- read.csv(file = 'data/R/df_pr_recall_for_R.csv', stringsAsFactors = TRUE)
df_pr_recall$story <- factor(df_pr_recall$story)
str(df_pr_recall)

df_p0r0 <- dplyr::filter(df_pr_recall, cond_amount=='p0/r0')
df_cpr <- dplyr::filter(df_pr_recall, cond_amount=='p/r')
df_recall <- dplyr::filter(df_pr_recall, cond_amount=='p_recall/r_recall')

'data.frame':	1360 obs. of  21 variables:
 $ sub                 : Factor w/ 36 levels "0116-1","0116-2",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ story               : Factor w/ 2 levels "1","2": 1 1 1 1 1 1 1 1 1 1 ...
 $ segment             : Factor w/ 22 levels "1_01","1_02",..: 1 2 2 2 3 4 4 4 5 6 ...
 $ segment_count       : num  0 1 1 1 2 3 3 3 4 5 ...
 $ segment_num         : int  1 2 2 2 3 4 4 4 5 6 ...
 $ base_segment        : Factor w/ 22 levels "1_01","1_02",..: 1 1 1 1 3 3 3 3 5 5 ...
 $ segment_pair        : Factor w/ 21 levels "1_01-02","1_02-03",..: 21 1 1 21 21 3 3 21 21 5 ...
 $ condition           : Factor w/ 8 levels "p","p0","p_r",..: 4 2 1 3 4 2 1 3 4 2 ...
 $ cond_direction      : Factor w/ 2 levels "b","f": 2 2 2 2 2 2 2 2 2 2 ...
 $ cond_amount         : Factor w/ 4 levels "p/r","p0/r0",..: 4 2 1 3 4 2 1 3 4 2 ...
 $ target_points       : num  4 7.5 7.5 7.5 6 8.5 8.5 8.5 8 7 ...
 $ e_total             : num  2 4.5 3.5 4.5 4 1 1.5 3.5 4 1 ...
 $ e_target            : num  

## target event hit rate

In [ ]:
# pr_recall_hit_ac proportion
pr_recall_hit_ac <- mixed(cbind(e_target, target_points - e_target) ~ 
                   cond_direction*cond_amount*segment_count_demean*story + 
                  (cond_direction*cond_amount|segment) + 
                  (cond_direction*cond_amount*segment_count_demean|sub), 
                df_pr_recall, family = binomial, method = "LRT", cl=cl)
pr_recall_hit_ac
saveRDS(pr_recall_hit_ac, "../pr_recall_hit_ac.rds")

In [ ]:
# hit_ac proportion seperate
p0r0_hit_ac <- mixed(cbind(e_target, target_points - e_target) ~ 
                   cond_direction*segment_count_demean*story + 
                  (cond_direction|segment) + 
                  (cond_direction*segment_count_demean|sub), 
                df_p0r0, family = binomial, method = "LRT", cl=cl)
p0r0_hit_ac
saveRDS(p0r0_hit_ac, "../p0r0_hit_ac.rds")

cpr_hit_ac <- mixed(cbind(e_target, target_points - e_target) ~ 
                   cond_direction*segment_count_demean*story + 
                  (cond_direction|segment) + 
                  (cond_direction*segment_count_demean|sub), 
                df_cpr, family = binomial, method = "LRT", cl=cl)
cpr_hit_ac
saveRDS(cpr_hit_ac, "../cpr_hit_ac.rds")

recall_hit_ac <- mixed(cbind(e_target, target_points - e_target) ~ 
                   cond_direction*segment_count_demean*story + 
                  (cond_direction|segment) + 
                  (cond_direction*segment_count_demean|sub), 
                df_recall, family = binomial, method = "LRT", cl=cl)
recall_hit_ac
saveRDS(recall_hit_ac, "../recall_hit_ac.rds")

## precision

In [24]:
# df_pr
df_pr <- read.csv(file = 'data/R/df_pr_for_R.csv', stringsAsFactors = TRUE)
df_pr$story <- factor(df_pr$story)
str(df_pr)

'data.frame':	981 obs. of  24 variables:
 $ sub                 : Factor w/ 36 levels "0116-1","0116-2",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ story               : Factor w/ 2 levels "1","2": 1 1 1 1 1 1 1 1 1 1 ...
 $ segment             : Factor w/ 22 levels "1_01","1_02",..: 2 2 2 4 4 4 6 6 6 8 ...
 $ segment_count       : num  1 1 1 3 3 3 5 5 5 7 ...
 $ segment_num         : int  2 2 2 4 4 4 6 6 6 8 ...
 $ base_segment        : Factor w/ 22 levels "1_01","1_02",..: 1 1 1 3 3 3 5 5 5 7 ...
 $ segment_pair        : Factor w/ 21 levels "1_01-02","1_02-03",..: 1 1 21 3 3 21 5 5 21 7 ...
 $ condition           : Factor w/ 6 levels "p","p0","p_r",..: 2 1 3 2 1 3 2 1 3 2 ...
 $ cond_direction      : Factor w/ 2 levels "b","f": 2 2 2 2 2 2 2 2 2 2 ...
 $ cond_amount         : Factor w/ 3 levels "p/r","p0/r0",..: 2 1 3 2 1 3 2 1 3 2 ...
 $ target_points       : num  7.5 7.5 7.5 8.5 8.5 8.5 7 7 7 5.5 ...
 $ e_total             : num  4.5 3.5 4.5 1 1.5 3.5 1 1 2 2 ...
 $ e_target            : num  1

In [ ]:
# pr_simi_other
pr_simi_other <- mixed(res_1_simi_other_z ~ cond_direction*cond_amount*segment_count_demean*story + 
                  (cond_direction*cond_amount|segment) + 
                  (cond_direction*cond_amount*segment_count_demean|sub), 
                  df_pr, method='S')
pr_simi_other
saveRDS(pr_simi_other, "../stats_models/pr_simi_other.rds")

In [ ]:
# simi_other seperate
p0r0_simi_other <- mixed(res_1_simi_other_z ~ cond_direction*segment_count_demean*story + 
                  (cond_direction|segment) + 
                  (cond_direction*segment_count_demean|sub), 
                  df_p0r0, method='S')
p0r0_simi_other
saveRDS(p0r0_simi_other, "../p0r0_simi_other.rds")

cpr_simi_other <- mixed(res_1_simi_other_z ~ cond_direction*segment_count_demean*story + 
                  (cond_direction|segment) + 
                  (cond_direction*segment_count_demean|sub), 
                  df_cpr, method='S')
cpr_simi_other
saveRDS(cpr_simi_other, "../cpr_simi_other.rds")

## convergence

In [ ]:
# pr_recall_MD
pr_recall_MD <- mixed(res_1_MD_sub_z ~ cond_direction*cond_amount*segment_count_demean*story + 
                     (cond_direction*cond_amount|segment) + 
                     (cond_direction*cond_amount*segment_count_demean|sub), 
                     df_pr_recall, method='S')
pr_recall_MD
saveRDS(pr_recall_MD, "../pr_recall_MD.rds")

In [ ]:
# consistency seperate
p0r0_MD <- mixed(res_1_MD_sub_z ~ cond_direction*segment_count_demean*story + 
                  (cond_direction|segment) + 
                  (cond_direction*segment_count_demean|sub), 
                  df_p0r0, method='S')
p0r0_MD
saveRDS(p0r0_MD, "../p0r0_MD.rds")

cpr_MD <- mixed(res_1_MD_sub_z ~ cond_direction*segment_count_demean*story + 
                  (cond_direction|segment) + 
                  (cond_direction*segment_count_demean|sub), 
                  df_cpr, method='S')
cpr_MD
saveRDS(cpr_MD, "../cpr_MD.rds")

recall_MD <- mixed(res_1_MD_sub_z ~ cond_direction*segment_count_demean*story + 
                  (cond_direction|segment) + 
                  (cond_direction*segment_count_demean|sub), 
                  df_recall, method='S')
recall_MD
saveRDS(recall_MD, "../recall_MD.rds")

# figure 5

In [ ]:
# p0r0_hit_econd_total poisson
df_p0r0$e_total_2 <- df_p0r0$e_total*2 # integers required

p0r0_hit_econd_total <- mixed(e_total_2 ~ cond_direction*segment_count_demean*story + 
                      (cond_direction|segment) + 
                      (cond_direction*segment_count_demean|sub), 
                    df_p0r0, family = poisson, method = "LRT", cl=cl)
p0r0_hit_econd_total
saveRDS(p0r0_hit_econd_total, "../p0r0_hit_econd_total.rds")

In [25]:
df_pr_econd <- read.csv(file = 'data/R/df_pr_econd_for_R.csv', stringsAsFactors = TRUE)
df_pr_econd$n_hits_2 <- df_pr_econd$n_hits*2 # integers required
df_pr_econd$story <- factor(df_pr_econd$story)
str(df_pr_econd)

df_p0r0_econd <- dplyr::filter(df_pr_econd, cond_amount=='p0/r0')
df_cpr_econd <- dplyr::filter(df_pr_econd, cond_amount=='p/r')

'data.frame':	3924 obs. of  15 variables:
 $ sub                 : Factor w/ 36 levels "0116-1","0116-2",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ story               : Factor w/ 2 levels "1","2": 1 1 1 1 1 1 1 1 1 1 ...
 $ segment             : Factor w/ 22 levels "1_01","1_02",..: 2 2 2 4 4 4 6 6 6 8 ...
 $ segment_count       : num  1 1 1 3 3 3 5 5 5 7 ...
 $ segment_num         : int  2 2 2 4 4 4 6 6 6 8 ...
 $ base_segment        : Factor w/ 22 levels "1_01","1_02",..: 1 1 1 3 3 3 5 5 5 7 ...
 $ segment_pair        : Factor w/ 21 levels "1_01-02","1_02-03",..: 1 1 21 3 3 21 5 5 21 7 ...
 $ condition           : Factor w/ 6 levels "p","p0","p_r",..: 2 1 3 2 1 3 2 1 3 2 ...
 $ cond_direction      : Factor w/ 2 levels "b","f": 2 2 2 2 2 2 2 2 2 2 ...
 $ cond_amount         : Factor w/ 3 levels "p/r","p0/r0",..: 2 1 3 2 1 3 2 1 3 2 ...
 $ e_cond              : Factor w/ 4 levels "e_ahead","e_gap",..: 3 3 3 3 3 3 3 3 3 3 ...
 $ n_hits              : num  1.5 1.5 1.5 0 0.5 2.5 0 0 0 2 ...
 $ segme

In [ ]:
p0r0_hit_econd <- mixed(n_hits_2 ~ cond_direction*segment_count_demean*story*e_cond + 
                  (cond_direction*e_cond|segment) + 
                  (cond_direction*segment_count_demean*e_cond|sub), 
                df_p0r0_econd, family = poisson, method = "LRT", cl=cl)
p0r0_hit_econd
saveRDS(p0r0_hit_econd, "../p0r0_hit_econd.rds")

## step plot

In [26]:
df_refer_long_p0r0 <- read.csv(file = 'data/R/df_refer_long_p0r0_for_R.csv', stringsAsFactors = TRUE)
df_refer_long_p0r0$refer_seg_e_2 <- df_refer_long_p0r0$refer_seg_e*2 # integers required
df_refer_long_p0r0$story <- factor(df_refer_long_p0r0$story)
str(df_refer_long_p0r0)

'data.frame':	3800 obs. of  14 variables:
 $ condition           : Factor w/ 2 levels "p0","r0": 2 2 2 2 2 2 2 2 2 2 ...
 $ story               : Factor w/ 2 levels "1","2": 2 2 2 2 2 1 1 1 1 1 ...
 $ sub                 : Factor w/ 36 levels "0116-1","0116-2",..: 1 1 1 1 1 2 2 2 2 2 ...
 $ segment_count       : num  1 3 5 7 9 1 3 5 7 9 ...
 $ segment_count_demean: num  -4.5 -2.5 -0.5 1.5 3.5 -4.5 -2.5 -0.5 1.5 3.5 ...
 $ main_or_gap         : Factor w/ 2 levels "gap","main": 2 2 2 2 2 2 2 2 2 2 ...
 $ base_segment        : Factor w/ 22 levels "1_01","1_02",..: 22 20 18 16 14 11 9 7 5 3 ...
 $ step_abs            : num  10 8 6 4 2 10 8 6 4 2 ...
 $ step_abs_demean     : num  6.75 4.75 2.75 0.75 -1.25 6.75 4.75 2.75 0.75 -1.25 ...
 $ refer_seg           : Factor w/ 42 levels "1_1.0","1_1.5",..: 22 22 22 22 22 1 1 1 1 1 ...
 $ base_refer_seg_pair : Factor w/ 440 levels "1_01_1_1.5","1_01_1_10.0",..: 421 381 341 301 261 201 161 121 81 41 ...
 $ refer_seg_e         : num  0 0 0 0 0 0 0 0 0

In [ ]:
p0_r0_hit_refer_step_2 <- mixed(refer_seg_e_2 ~ condition*main_or_gap*step_abs_demean*story + 
                           (condition|base_segment) + 
                           (1|base_refer_seg_pair) +
                           (condition*main_or_gap*step_abs_demean*story|sub), 
                       df_refer_long_p0r0, family = poisson, method = "LRT", cl=cl)
p0_r0_hit_refer_step_2
saveRDS(p0_r0_hit_refer_step_2, "../p0_r0_hit_refer_step_2.rds")

# figure 6

In [27]:
df_seg_seg_type_p0r0 <- read.csv(file = 'data/R/df_seg_seg_type_p0r0_for_R.csv', stringsAsFactors = TRUE)
df_seg_seg_type_p0r0$story <- factor(df_seg_seg_type_p0r0$story)
str(df_seg_seg_type_p0r0)

'data.frame':	4415 obs. of  19 variables:
 $ sub                     : Factor w/ 36 levels "0116-1","0116-2",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ condition               : Factor w/ 2 levels "p0","r0": 1 1 1 1 1 1 1 1 1 1 ...
 $ story                   : Factor w/ 2 levels "1","2": 1 1 1 1 1 1 1 1 1 1 ...
 $ segment                 : Factor w/ 22 levels "1_01","1_02",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ segment_count           : num  1 1 1 1 1 1 1 1 1 1 ...
 $ base_seg_num            : int  1 1 1 1 1 1 1 1 1 1 ...
 $ base_segment            : Factor w/ 22 levels "1_01","1_02",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ ref_seg_num             : num  1.5 1.5 2 2 2.5 3 3.5 4 4.5 5 ...
 $ ref_step_base           : num  0.5 0.5 1 1 1.5 2 2.5 3 3.5 4 ...
 $ ref_step_base_abs       : num  0.5 0.5 1 1 1.5 2 2.5 3 3.5 4 ...
 $ ref_seg_type            : Factor w/ 4 levels "ahead_gap","ahead_main",..: 3 3 4 4 1 2 1 2 1 2 ...
 $ main_or_gap             : Factor w/ 2 levels "gap","main": 1 1 2 2 1 2 1 2 1 2 ...
 $ refer_t

In [ ]:
p0_r0_seg_seg_type_1 <- mixed(cbind(total_hits, total_points - total_hits) ~ 
                              condition*refer_type_merge*main_or_gap + ref_step_base_abs_demean + 
                              (condition|base_segment) + 
                              (1|base_refer_seg_pair) +
                              (condition*refer_type_merge*main_or_gap + ref_step_base_abs_demean|sub), 
                          df_seg_seg_type_p0r0, family = binomial, method = "LRT", cl=cl)
p0_r0_seg_seg_type_1
saveRDS(p0_r0_seg_seg_type_1, "../p0_r0_seg_seg_type_1.rds")

# figure 7

In [31]:
df_seg_seg_boost_type_p0r0 <- read.csv(file = 'data/R/df_seg_seg_boost_type_p0r0_for_R.csv', stringsAsFactors = TRUE)
df_seg_seg_boost_type_p0r0$story <- factor(df_seg_seg_boost_type_p0r0$story)
str(df_seg_seg_boost_type_p0r0)

'data.frame':	4415 obs. of  20 variables:
 $ sub                     : Factor w/ 36 levels "0116-1","0116-2",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ condition               : Factor w/ 2 levels "p0","r0": 1 1 1 1 1 1 1 1 1 1 ...
 $ story                   : Factor w/ 2 levels "1","2": 1 1 1 1 1 1 1 1 1 1 ...
 $ segment                 : Factor w/ 22 levels "1_01","1_02",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ segment_count           : num  1 1 1 1 1 1 1 1 1 1 ...
 $ base_segment            : Factor w/ 22 levels "1_01","1_02",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ base_seg_num            : int  1 1 1 1 1 1 1 1 1 1 ...
 $ ref_seg_num             : num  1.5 1.5 2 2 2.5 3 3.5 4 4.5 5 ...
 $ ref_step_base           : num  0.5 0.5 1 1 1.5 2 2.5 3 3.5 4 ...
 $ ref_step_base_abs       : num  0.5 0.5 1 1 1.5 2 2.5 3 3.5 4 ...
 $ ref_seg_type            : Factor w/ 4 levels "ahead_gap","ahead_main",..: 3 3 4 4 1 2 1 2 1 2 ...
 $ main_or_gap             : Factor w/ 2 levels "gap","main": 1 1 2 2 1 2 1 2 1 2 ...
 $ refer_b

In [ ]:
p0_r0_seg_seg_boost_type_tri_1 <- mixed(cbind(total_hits, total_points - total_hits) ~ 
                              condition*refer_boost_type_tri*main_or_gap + ref_step_base_abs_demean + 
                              (condition|base_segment) + 
                              (1|base_refer_seg_pair) +
                              (condition*refer_boost_type_tri*main_or_gap + ref_step_base_abs_demean|sub), 
                          df_seg_seg_boost_type_p0r0, family = binomial, method = "LRT", cl=cl)
p0_r0_seg_seg_boost_type_tri_1
saveRDS(p0_r0_seg_seg_boost_type_tri_1, "../p0_r0_seg_seg_boost_type_tri_1.rds")

## figure 8

In [4]:
df_seg_seg_rr_type_p0r0_main <- read.csv(file = '../data/R/df_seg_seg_rr_type_p0r0_main_for_R.csv', stringsAsFactors = TRUE)
df_seg_seg_rr_type_p0r0_main$story <- factor(df_seg_seg_rr_type_p0r0_main$story)
str(df_seg_seg_rr_type_p0r0_main)

'data.frame':	2496 obs. of  19 variables:
 $ sub                     : Factor w/ 36 levels "0116-1","0116-2",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ condition               : Factor w/ 2 levels "p0","r0": 1 1 1 1 1 1 1 1 1 1 ...
 $ story                   : Factor w/ 2 levels "1","2": 1 1 1 1 1 1 1 1 1 1 ...
 $ segment                 : Factor w/ 22 levels "1_01","1_02",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ segment_count           : int  1 1 1 1 1 1 1 1 1 1 ...
 $ base_seg_num            : int  1 1 1 1 1 1 1 1 1 1 ...
 $ base_segment            : Factor w/ 22 levels "1_01","1_02",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ ref_seg_num             : num  2 2 3 4 5 6 7 8 9 10 ...
 $ ref_step_base           : num  1 1 2 3 4 5 6 7 8 9 ...
 $ ref_step_base_abs       : num  1 1 2 3 4 5 6 7 8 9 ...
 $ ref_seg_type            : Factor w/ 2 levels "ahead_main","target": 2 2 1 1 1 1 1 1 1 1 ...
 $ main_or_gap             : Factor w/ 1 level "main": 1 1 1 1 1 1 1 1 1 1 ...
 $ rr_type                 : Factor w/ 3 levels "non

In [ ]:
p0_r0_seg_seg_rr_type_1 <- mixed(cbind(total_hits, total_points - total_hits) ~ 
                               condition*rr_type + ref_step_base_abs_demean + 
                              (condition|base_segment) + 
                              (1|base_refer_seg_pair) +
                              (condition*rr_type + ref_step_base_abs_demean|sub), 
                              df_seg_seg_rr_type_p0r0_main, family = binomial, method = "LRT", cl=cl)
p0_r0_seg_seg_rr_type_1
saveRDS(p0_r0_seg_seg_rr_type_1, "../p0_r0_seg_seg_rr_type_1.rds")